# Installing & Imports

In [146]:
conda install pandas==0.24.2

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - pandas==0.24.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         153 KB
    ------------------------------------------------------------
                                           Total:         153 KB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2019.11.~ --> pkgs/main::ca-certificates-2020.1.1-0
  openssl            conda-forge::openssl-1.1.1d-h516909a_0 --> pkgs/main::openssl-1.1.1d-h7b6447c_4

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                       conda-forge --> pkgs/main



certifi-2019.11.28   | 153 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying

In [147]:
conda upgrade --all -y

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [148]:
conda install snorkel==0.9.0 -c conda-forge

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - snorkel==0.9.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         149 KB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2019.11.28-hecc5488_0
  certifi                                         pkgs/main --> conda-forge
  openssl              pkgs/main::openssl-1.1.1d-h7b6447c_4 --> conda-forge::openssl-1.1.1d-h516909a_0



certifi-2019.11.28   | 149 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction

In [149]:
conda install -c conda-forge textblob

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [23]:
from snorkel.labeling import LFAnalysis
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier,LabelModel
from snorkel.preprocess import preprocessor
from textblob import TextBlob
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
from csv import writer
import re
import pickle
import time
import json

# Web Scraping

## Browse all from DVD releases page

In [24]:
main = 'https://www.rottentomatoes.com/api/private/v2.0/browse?maxTomato=100&services=amazon%3Bhbo_go%3Bitunes%3Bnetflix_iw%3Bvudu%3Bamazon_prime%3Bfandango_now&certified&sortBy=release&type=dvd-streaming-all&page='

In [25]:
# Get movie url
movie_url = []
start_page = 2 ; end_page = 19
while start_page <= end_page:
#     time.sleep(7)
    url = main + str(start_page)
    response = requests.get(url)
    if response.status_code !=200:
        print('Request error')
        break
    file = json.loads(response.text)
    for i in file['results']:
        movie_url = movie_url + [i['url']]
    start_page +=1

In [26]:
print('Examples for the url:\n')
for i in range(3):
    print(movie_url[i])
print('\nNumber of movies in list: {}'.format(len(movie_url)))

Examples for the url:

/m/a_shaun_the_sheep_movie_farmageddon
/m/my_name_is_myeisha
/m/high_strung_free_dance

Number of movies in list: 32


In [29]:
# Split into lists of 50 movies to do the scraping
movie_url_split = [movie_url[i:i+50] for i in range(0,600,50)]

In [28]:
len(movie_url_split)

12

In [30]:
# Get reviews from the web
reviews = []
titles = []
ratings = []
for split in movie_url_split: # Loop through each split
    time.sleep(7)
    for title in split: # Loop through each movie title
        url = 'https://www.rottentomatoes.com'+title
        time.sleep(7)
        response = requests.get(url)
        # Check the request status code
        if response.status_code != 200:
            print('Request error')
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Get labels from each review (fresh vs. rotten)
        fresh_rotten = soup.find_all(class_="review_quote")
        
        # Get movie title
        title = soup.find(class_="mop-ratings-wrap__title mop-ratings-wrap__title--top").getText()
        
        # Get reviews
        review = soup.find_all('blockquote')
        for i in review:
            j = str(i.contents[1])
            j = j.replace("<p>\n                    \n                        ","")
            j = j.replace("\n                    \n                </p>","")
            reviews = reviews + [j]
            titles = titles + [title]
        
        # Identify labels
        for i in fresh_rotten:
            temp = str(i.findChildren()[2])
            if re.search('rotten',temp):
                ratings = ratings + ['rotten']
            else:
                ratings = ratings + ['fresh']
            

In [34]:
# Create the DataFrame to store the scraped data
df = pd.DataFrame([titles,reviews,ratings],index = ['title','review','rating']).T

In [35]:
# Clean the data (drop duplicates, check missing values etc.)
df = df.drop_duplicates()
df = df.replace([None],np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 3 columns):
title     270 non-null object
review    270 non-null object
rating    270 non-null object
dtypes: object(3)
memory usage: 8.4+ KB


In [36]:
df.dropna(inplace=True)
df.head(3)

,title,review,rating
0,A Shaun the Sheep Movie: Farmageddon,"Farmageddon made me laugh quite a few times, a...",fresh
1,A Shaun the Sheep Movie: Farmageddon,That all these characters and then some have d...,fresh
2,A Shaun the Sheep Movie: Farmageddon,"It's a very funny movie - and an endlessly, re...",fresh


In [ ]:
# Export to CSV files
# df.to_csv('web_scraping_rotten_tomatoes_part2.csv')

# Reading & Preparing TSV file 

In [151]:
# Read TSV file
tsv_reviews = pd.read_csv('/project/reviews.tsv', sep='\t', header=0, encoding='unicode_escape')

In [152]:
tsv_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [153]:
# Extract review and fresh columns
tsv_reviews = pd.DataFrame(tsv_reviews, columns = ['review', 'fresh'])

In [154]:
tsv_reviews.head()

,review,fresh
0,A distinctly gallows take on contemporary fina...,fresh
1,It's an allegory in search of a meaning that n...,rotten
2,... life lived in a bubble in financial dealin...,fresh
3,Continuing along a line introduced in last yea...,fresh
4,... a perverse twist on neorealism...,fresh


In [155]:
tsv_reviews.isnull().sum()

review    5563
fresh        0
dtype: int64

In [156]:
# drop NaN rows in reviews
index_name = tsv_reviews[(tsv_reviews['review'].isnull())].index
tsv_reviews.drop(index_name, inplace= True)

In [157]:
tsv_reviews.isnull().sum()

review    0
fresh     0
dtype: int64

In [158]:
# rename fresh as 1 and rotten as 0
tsv_reviews['fresh'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

In [159]:
#Rename columns
tsv_reviews.rename(columns={'fresh':'Freshness','review':'Review'},inplace=True)
tsv_reviews = tsv_reviews.sample(5000)
#take 5000
tsv_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 48887 to 10927
Data columns (total 2 columns):
Review       5000 non-null object
Freshness    5000 non-null object
dtypes: object(2)
memory usage: 117.2+ KB


# Reading & Preparing CSV file 

In [160]:
# Read CSV file
csv_reviews= pd.read_csv('/project/rotten_tomatoes_reviews.csv')
csv_reviews.head()


,Freshness,Review
0,1,"Manakamana doesn't answer any questions, yet ..."
1,1,Wilfully offensive and powered by a chest-thu...
2,0,It would be difficult to imagine material mor...
3,0,Despite the gusto its star brings to the role...
4,0,If there was a good idea at the core of this ...


In [161]:
#Swap Freshness and Review 
columns_titles = ["Review","Freshness"]
csv_reviews=csv_reviews.reindex(columns=columns_titles)
csv_reviews = csv_reviews.sample(5000)

csv_reviews.head()
csv_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 242759 to 224039
Data columns (total 2 columns):
Review       5000 non-null object
Freshness    5000 non-null int64
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


# Web Scraping & Preparing scrapped data 

In [162]:
#Read Web Scraping Data
web_scraping_reviews= pd.read_csv('/project/web_scraping_rotten_tomatoes_part2.csv')
web_scraping_reviews.head()

web_scraping_reviews = web_scraping_reviews.sample(5000)

In [163]:
# rename fresh as 1 and rotten as 0
web_scraping_reviews['rating'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

#Rename Rating to Review 
web_scraping_reviews.rename(columns={'rating':'Freshness', 'review':'Review'},inplace=True)
web_scraping_reviews.head()

# Extract Review and Freshness columns
web_scraping_reviews= pd.DataFrame(web_scraping_reviews, columns = ['Review', 'Freshness'])

In [164]:
web_scraping_reviews.head()

,Review,Freshness
2528,"EMPATHY, INC. mirrors a breed of low-fi fantas...",1
683,Lawyer Cohn apparently pulled many malicious s...,1
171,"Shallow, asinine and tedious.",0
1966,"With few exceptions, the post-REC Spanish horr...",0
4586,Touchdown for faith-based fans; underage drink...,1


# Combining all the data together 

In [165]:
# csv_reviews.info()
# tsv_reviews.info()
# web_scraping_reviews.info()



# Concat two files into all_reviews
all_reviews=pd.concat([csv_reviews, tsv_reviews,web_scraping_reviews],axis=0, sort=False)
all_reviews.head()

,Review,Freshness
242759,"Edgy Barrymore romcom has lots of sex, language.",1
469706,Boogie Nights is a shockingly well-made film.,1
203932,"Birdsong is remarkable. Unsettling, visceral,...",1
272716,A surprisingly sweet and funny ode to male fr...,1
285174,Once again proof that life is rough after Sat...,0


In [166]:
all_reviews.shape

(15000, 2)

# Split into test and training set 

In [167]:
from sklearn.model_selection import train_test_split

In [199]:
all_reviews['Freshness'] = all_reviews['Freshness'].astype(int)

In [203]:
train, test = train_test_split(all_reviews,test_size=0.2,stratify = all_reviews['Freshness'])

In [204]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 115655 to 27893
Data columns (total 2 columns):
Review       12000 non-null object
Freshness    12000 non-null int64
dtypes: int64(1), object(1)
memory usage: 281.2+ KB


In [205]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 22097 to 165024
Data columns (total 2 columns):
Review       3000 non-null object
Freshness    3000 non-null int64
dtypes: int64(1), object(1)
memory usage: 70.3+ KB


In [171]:
train.head()

,Review,Freshness
1710,The film lacks a direction and instead feels l...,0
4399,Dumbo is a lot of work by a lot of talented pe...,1
2567,It is a fantastic idea that never gels into an...,0
2365,If you want some laughs and a bit of insight i...,0
61278,An endless series of scatological jokes saps ...,0


In [172]:
#get rid off training labels 
train = train.drop('Freshness', 1)

In [206]:
test['Freshness'].value_counts()

1    1776
0    1224
Name: Freshness, dtype: int64

In [207]:
#From labelled test set, extract a sample to find out about which labelling functions could be written
#Not sure how big the development split_ can be --> take sample of 1000 data points 

development_split = test.sample(1000,random_state=42)
development_split.head()


,Review,Freshness
41101,"Vibrant, visually exciting and emotionally res...",1
18134,An enjoyable campy bad film.,0
3215,"This brave little film, a tale of an uncompreh...",1
222,"Excellent, accurate, smart analysis of sexual ...",1
181403,a merger of Cold Weather's mumblenoir and LA ...,1


In [208]:
#For finding labelling functions: 
development_split

,Review,Freshness
41101,"Vibrant, visually exciting and emotionally res...",1
18134,An enjoyable campy bad film.,0
3215,"This brave little film, a tale of an uncompreh...",1
222,"Excellent, accurate, smart analysis of sexual ...",1
181403,a merger of Cold Weather's mumblenoir and LA ...,1
3906,This film is basically a well-meaning fable th...,1
1563,The chemistry between the main trio is very pl...,1
654,An engaging personal essay documentary about n...,1
428143,"If nothing else, The Wolverine proves how cha...",1
20559,A noirish thriller with experimental trimmings...,1


In [209]:
development_split.count()

Review       1000
Freshness    1000
dtype: int64

In [217]:
development_split.to_csv('development_split.csv')

In [218]:
development_split.count()

Review       1000
Freshness    1000
dtype: int64

In [219]:
development_split.head()

,Review,Freshness
41101,"Vibrant, visually exciting and emotionally res...",1
18134,An enjoyable campy bad film.,0
3215,"This brave little film, a tale of an uncompreh...",1
222,"Excellent, accurate, smart analysis of sexual ...",1
181403,a merger of Cold Weather's mumblenoir and LA ...,1


In [220]:
#Might have to get rid off index?

development_split[development_split['Freshness'] !=1].count()

Review       430
Freshness    430
dtype: int64

In [221]:
development_split['Freshness'].value_counts()

1    570
0    430
Name: Freshness, dtype: int64